### Dados Macroeconômicos Seção 4 (01/1999 a 12/2024)

* PIB (Trimestral) - IBGE - sidrapy - Tabela 1621
* IPCA (Mensal)	- BCB - python-bcb - SGS 433
* IPCA (12 meses) - BCB - python-bcb - SGS 13522
* Selic Meta - BCB - python-bcb - SGS 432
* Expectativas - BCB - python-bcb - Endpoint Expectativas - Filtrar por Indicador='IPCA'

In [1]:
# !pip install python-bcb pandas matplotlib seaborn sidrapy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bcb import sgs
from bcb import Expectativas
import sidrapy
import datetime

# Configurações de Estilo
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Parâmetros Globais
DATA_INICIAL = '1999-01-01'
DATA_FINAL = '2024-12-31'

# 2. Coleta de Dados via SGS (Sistema Gerenciador de Séries)
Coleta das séries de IPCA e Selic diretamente do Banco Central utilizando o módulo `sgs`.
- **433**: IPCA Mensal (%)
- **13522**: IPCA Acumulado 12 meses (%)
- **432**: Meta Selic (% a.a.)

In [3]:
from pathlib import Path

def _encontrar_raiz_projeto() -> Path:
    cwd = Path.cwd().resolve()
    for p in (cwd, *cwd.parents):
        if (p / 'dados').exists():
            return p
    return cwd


def _to_float_misto_ptbr(x):
    import numpy as np

    if x is None:
        return np.nan
    s = str(x).strip()
    if not s or s.lower() == 'nan':
        return np.nan

    # Trata números em formato pt-BR (ex: 15,00) e mistos (ex: 1.234,56)
    if ',' in s and '.' in s:
        s = s.replace('.', '').replace(',', '.')
    elif ',' in s:
        s = s.replace(',', '.')

    try:
        return float(s)
    except ValueError:
        return np.nan


def _coleta_sgs_fallback_local(data_inicial: str, data_final: str, pasta_dados_rel: str = 'dados/rafaela') -> pd.DataFrame:
    """Fallback quando o SGS (api.bcb.gov.br) está bloqueado (ex: HTTP 403).

    Usa arquivos locais existentes no repositório:
    - IPCA15_mensal.csv (proxy para inflação mensal/12m)
    - Taxa_selic_aa.csv (Meta Selic por vigência -> série diária)
    """
    raiz = _encontrar_raiz_projeto()
    pasta = (raiz / pasta_dados_rel).resolve()

    ipca_path = pasta / 'IPCA15_mensal.csv'
    selic_path = pasta / 'Taxa_selic_aa.csv'

    dfs = []
    data_inicial_dt = pd.to_datetime(data_inicial)
    data_final_dt = pd.to_datetime(data_final)

    # IPCA (fallback via IPCA-15)
    if ipca_path.exists():
        df_ipca = pd.read_csv(ipca_path, sep=';')
        mapa_mes = {
            'JAN': 1, 'FEV': 2, 'MAR': 3, 'ABR': 4, 'MAI': 5, 'JUN': 6,
            'JUL': 7, 'AGO': 8, 'SET': 9, 'OUT': 10, 'NOV': 11, 'DEZ': 12,
        }
        df_ipca['MesNum'] = df_ipca['MÊS'].astype(str).str.strip().str.upper().map(mapa_mes)
        df_ipca = df_ipca.dropna(subset=['MesNum'])
        df_ipca['Data'] = pd.to_datetime(
            df_ipca['ANO'].astype(int).astype(str)
            + '-'
            + df_ipca['MesNum'].astype(int).astype(str)
            + '-01'
        )
        df_ipca = df_ipca.set_index('Data').sort_index()
        df_ipca = df_ipca.rename(columns={
            'VAR_PERC_MES': 'IPCA_Mensal',
            'VAR_PERC_12_MESES': 'IPCA_12m',
        })
        df_ipca['IPCA_Mensal'] = pd.to_numeric(df_ipca['IPCA_Mensal'], errors='coerce')
        df_ipca['IPCA_12m'] = pd.to_numeric(df_ipca['IPCA_12m'], errors='coerce')
        df_ipca = df_ipca[['IPCA_Mensal', 'IPCA_12m']]
        dfs.append(df_ipca)
    else:
        print(f"Fallback local: arquivo não encontrado: {ipca_path}")

    # Selic Meta (fallback via arquivo de vigências)
    if selic_path.exists():
        df_selic_raw = pd.read_csv(selic_path, sep=';', dtype=str)
        df_selic_raw['META_SELIC_aa'] = df_selic_raw['META_SELIC_aa'].apply(_to_float_misto_ptbr)

        vig = df_selic_raw['VIGENCIA'].fillna('')
        partes = vig.str.split('-', n=1, expand=True)
        inicio_str = partes[0].astype(str).str.strip()
        fim_str = partes[1].astype(str).str.strip() if partes.shape[1] > 1 else ''

        df_selic_raw['Inicio'] = pd.to_datetime(inicio_str, dayfirst=True, errors='coerce')
        df_selic_raw['Fim'] = pd.to_datetime(fim_str, dayfirst=True, errors='coerce')

        series_parts = []
        for row in df_selic_raw.itertuples(index=False):
            start = getattr(row, 'Inicio')
            end = getattr(row, 'Fim')
            val = getattr(row, 'META_SELIC_aa')

            if pd.isna(start) or pd.isna(val):
                continue
            if pd.isna(end):
                end = data_final_dt
            if end < start:
                continue

            rng = pd.date_range(start, end, freq='D')
            series_parts.append(pd.Series(val, index=rng))

        if series_parts:
            selic = pd.concat(series_parts).sort_index()
            selic = selic[~selic.index.duplicated(keep='last')]
            df_selic = selic.to_frame('Selic_Meta')
            dfs.append(df_selic)
        else:
            print("Fallback local: não foi possível construir a série diária da Selic a partir das vigências.")
    else:
        print(f"Fallback local: arquivo não encontrado: {selic_path}")

    if not dfs:
        return pd.DataFrame()

    df = pd.concat(dfs, axis=1).sort_index()
    df = df[(df.index >= data_inicial_dt) & (df.index <= data_final_dt)]
    return df


def coleta_dados_sgs():
    print("Iniciando coleta de dados do SGS...")
    # O formato correto para sgs.get é {'Nome_da_Coluna': Codigo_SGS}
    codigos = {
        'IPCA_Mensal': 433,
        'IPCA_12m': 13522,
        'Selic_Meta': 432
    }

    dfs = []
    start_date = pd.to_datetime(DATA_INICIAL)
    end_date = pd.to_datetime(DATA_FINAL)

    # Loop em janelas para reduzir risco de timeouts
    while start_date <= end_date:
        window_end = start_date + pd.DateOffset(years=5)
        if window_end > end_date:
            window_end = end_date

        print(f"Coletando chunk: {start_date.date()} até {window_end.date()}")

        try:
            # Força datetime/date simples (melhor compatibilidade)
            df_chunk = sgs.get(codigos, start=start_date.date(), end=window_end.date())
            if not df_chunk.empty:
                dfs.append(df_chunk)
        except Exception as e:
            print(f"Erro na coleta do chunk {start_date.date()}: {e}")

        start_date = window_end + pd.Timedelta(days=1)

    if dfs:
        df = pd.concat(dfs)
        df = df[~df.index.duplicated(keep='first')]
        print(f"Dados SGS coletados: {df.shape[0]} registros.")
        return df

    print("SGS indisponível (ex: HTTP 403 'Acesso Negado'). Tentando fallback local em dados/rafaela...")
    df_local = _coleta_sgs_fallback_local(DATA_INICIAL, DATA_FINAL)
    if df_local.empty:
        print("Fallback local também falhou: retornando DataFrame vazio.")
    else:
        print(f"Dados locais carregados: {df_local.shape[0]} registros.")
    return df_local


df_sgs = coleta_dados_sgs()
df_sgs.head(), df_sgs.tail()

Iniciando coleta de dados do SGS...
Coletando chunk: 1999-01-01 até 2004-01-01
Erro na coleta do chunk 1999-01-01: Download error: code = 433
Coletando chunk: 2004-01-02 até 2009-01-02
Erro na coleta do chunk 2004-01-02: Download error: code = 433
Coletando chunk: 2009-01-03 até 2014-01-03
Erro na coleta do chunk 2009-01-03: Download error: code = 433
Coletando chunk: 2014-01-04 até 2019-01-04
Erro na coleta do chunk 2014-01-04: Download error: code = 433
Coletando chunk: 2019-01-05 até 2024-01-05
Erro na coleta do chunk 2019-01-05: Download error: code = 433
Coletando chunk: 2024-01-06 até 2024-12-31
Erro na coleta do chunk 2024-01-06: Download error: code = 433
SGS indisponível (ex: HTTP 403 'Acesso Negado'). Tentando fallback local em dados/rafaela...
Dados locais carregados: 9492 registros.


(            IPCA_Mensal  IPCA_12m  Selic_Meta
 1999-01-01         0.68       1.8        29.0
 1999-01-02          NaN       NaN        29.0
 1999-01-03          NaN       NaN        29.0
 1999-01-04          NaN       NaN        29.0
 1999-01-05          NaN       NaN        29.0,
             IPCA_Mensal  IPCA_12m  Selic_Meta
 2024-12-27          NaN       NaN       12.25
 2024-12-28          NaN       NaN       12.25
 2024-12-29          NaN       NaN       12.25
 2024-12-30          NaN       NaN       12.25
 2024-12-31          NaN       NaN       12.25)

# 3. Coleta de Dados do PIB (IBGE)
Coleta da série do PIB trimestral utilizando a biblioteca `sidrapy`.
- **Tabela 1621**: Série encadeada do índice de volume trimestral com ajuste sazonal.
- **Variável 584**: Índice de volume trimestral com ajuste sazonal.
- **Setor**: PIB a preços de mercado.

In [ ]:
def coleta_pib():
    print("Iniciando coleta de dados do PIB (IBGE)...")
    # Tabela 1621: Série encadeada do índice de volume trimestral com ajuste sazonal
    # Variável 584: Série encadeada do índice de volume trimestral com ajuste sazonal (Base: média de 1995 = 100)
    # Classificação 11255: Setores de atividade -> 90707: PIB a preços de mercado
    
    try:
        pib = sidrapy.get_table(
            table_code="1621",
            territorial_level="1",
            ibge_territorial_code="all",
            variable="584",
            classification="11255/90707",
            period="all"
        )
        
        # Limpeza e tratamento
        if pib.empty:
            print("Retorno vazio do sidrapy.")
            return pd.DataFrame()
            
        pib = pib.iloc[1:] # Remove a linha de cabeçalho/descrição
        pib = pib[['V', 'D2C']] # V: Valor, D2C: Trimestre (ex: 199601)
        pib.columns = ['PIB_Indice', 'Trimestre']
        
        # Converter valor para numérico
        pib['PIB_Indice'] = pd.to_numeric(pib['PIB_Indice'])
        
        # Converter Trimestre para Data (Primeiro dia do trimestre)
        # Formato IBGE: YYYY0Q (ex: 199901 -> 1º tri 1999)
        # 01 -> Mês 01, 02 -> Mês 04, 03 -> Mês 07, 04 -> Mês 10
        pib['Ano'] = pib['Trimestre'].str[:4]
        pib['Tri'] = pib['Trimestre'].str[-1].astype(int)
        pib['Mes'] = (pib['Tri'] * 3) - 2
        pib['Data'] = pd.to_datetime(pib['Ano'] + '-' + pib['Mes'].astype(str) + '-01')
        
        pib = pib.set_index('Data').sort_index()
        
        # Filtrar período
        pib = pib[(pib.index >= DATA_INICIAL) & (pib.index <= DATA_FINAL)]
        
        # Rebase para 1999=100 (Média de 1999 = 100)
        if not pib.empty:
            base_1999 = pib[pib.index.year == 1999]['PIB_Indice'].mean()
            pib['PIB_Indice'] = (pib['PIB_Indice'] / base_1999) * 100
        
        print(f"Dados PIB coletados: {pib.shape[0]} registros trimestrais.")
        return pib[['PIB_Indice']]
        
    except Exception as e:
        print(f"Erro na coleta do PIB: {e}")
        return pd.DataFrame()

df_pib = coleta_pib()
df_pib.head()

# 4. Coleta de Expectativas de Inflação (Focus)
Coleta das expectativas de mercado para o IPCA anual utilizando o módulo `Expectativas`.
O processo envolve:
1. Baixar dados do endpoint `ExpectativasMercadoAnuais`.
2. Filtrar pelo indicador IPCA.
3. Selecionar apenas as expectativas para o **ano corrente** (onde `DataReferencia` é igual ao ano da data de coleta).
4. Realizar uma reamostragem mensal calculando a média das expectativas diárias.

In [ ]:
def coleta_expectativas():
    print("Iniciando coleta de Expectativas Focus...")
    em = Expectativas()
    ep = em.get_endpoint('ExpectativasMercadoAnuais')
    
    # Filtra por IPCA e data inicial para otimizar a consulta
    try:
        df_focus = ep.query().filter(ep.Indicador == 'IPCA').filter(ep.Data >= DATA_INICIAL).collect()
        
        # Converter Data para datetime
        df_focus['Data'] = pd.to_datetime(df_focus['Data'])
        df_focus['DataReferencia'] = df_focus['DataReferencia'].astype(int)
        
        # Filtrar período de análise (garantia adicional)
        df_focus = df_focus[(df_focus['Data'] >= DATA_INICIAL) & (df_focus['Data'] <= DATA_FINAL)]
        
        # Filtrar expectativa para o ano corrente
        # Lógica: Queremos saber qual era a expectativa para 2023 em 2023, etc.
        df_focus = df_focus[df_focus['DataReferencia'] == df_focus['Data'].dt.year]
        
        # Selecionar colunas e renomear
        df_focus = df_focus[['Data', 'Media']]
        df_focus = df_focus.rename(columns={'Media': 'Expectativa_IPCA_AnoCorrente'})
        
        # Reamostragem mensal (Média mensal das expectativas diárias)
        # 'ME' é o alias para Month End no pandas mais recente. Use 'M' se der erro em versões antigas.
        df_focus = df_focus.set_index('Data').resample('ME').mean()
        
        print(f"Dados Focus processados: {df_focus.shape[0]} registros mensais.")
        return df_focus
        
    except Exception as e:
        print(f"Erro na coleta do Focus: {e}")
        return pd.DataFrame()

df_focus = coleta_expectativas()
df_focus.head(), df_focus.tail()

# 5. Tratamento e Consolidação
Unificação das bases de dados do SGS, PIB e Focus em um único DataFrame com frequência mensal.
Realiza-se o alinhamento das datas (início do mês) e tratamento de dados faltantes.
Para o PIB (trimestral), os dados serão repetidos ou interpolados para frequência mensal (Forward Fill).

In [ ]:
def consolidar_dados(df_sgs, df_focus, df_pib):
    print("Consolidando dados...")
    
    # Ajuste de índice do Focus para o primeiro dia do mês para alinhar com SGS
    df_focus_adj = df_focus.copy()
    df_focus_adj.index = df_focus_adj.index.to_period('M').to_timestamp()
    
    # Merge SGS e Focus
    df_final = pd.merge(df_sgs, df_focus_adj, left_index=True, right_index=True, how='outer')
    
    # Merge PIB
    # O PIB é trimestral. Ao fazer o merge, teremos NaNs nos meses sem dado.
    # Vamos usar forward fill para preencher os meses dentro do trimestre (ou interpolação linear se preferir)
    # Aqui usaremos ffill para manter o valor do trimestre até o próximo.
    df_final = pd.merge(df_final, df_pib, left_index=True, right_index=True, how='outer')
    
    # Filtro final de datas para garantir o range solicitado
    df_final = df_final[(df_final.index >= DATA_INICIAL) & (df_final.index <= DATA_FINAL)]
    
    # Preenchimento de dados faltantes
    # Primeiro ffill para preencher PIB e eventuais buracos pequenos
    df_final = df_final.ffill()
    
    print(f"Dataset final consolidado: {df_final.shape}")
    return df_final

df_economico = consolidar_dados(df_sgs, df_focus, df_pib)
df_economico.tail()

# 6. Visualização Exploratória
Geração de gráficos para análise visual das séries temporais.
- **Gráfico 1**: Comparação entre IPCA acumulado em 12 meses e a Meta Selic.
- **Gráfico 2**: Evolução das expectativas de inflação para o ano corrente.
- **Gráfico 3**: Evolução do PIB (Índice de Volume).

In [ ]:
import matplotlib.dates as mdates
import matplotlib.patches as mpatches

def plotar_dados(df):
    fig, axes = plt.subplots(3, 1, figsize=(14, 15), sharex=True)
    
    # Gráfico 1: IPCA 12m vs Selic
    sns.lineplot(data=df, x=df.index, y='IPCA_12m', ax=axes[0], label='IPCA Acumulado 12m', color='#1f77b4', linewidth=2)
    sns.lineplot(data=df, x=df.index, y='Selic_Meta', ax=axes[0], label='Meta Selic', color='#d62728', linestyle='--', linewidth=2)
    
    axes[0].set_title('Dinâmica da Inflação e Taxa de Juros (1999-2024)', fontsize=16, fontweight='bold')
    axes[0].set_ylabel('% a.a.', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico 2: Expectativas
    sns.lineplot(data=df, x=df.index, y='Expectativa_IPCA_AnoCorrente', ax=axes[1], label='Focus', color='#2ca02c', linewidth=2)
    
    axes[1].set_title('Expectativas de Inflação (IPCA ano corrente)', fontsize=16, fontweight='bold')
    axes[1].set_ylabel('%', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    # Gráfico 3: PIB
    sns.lineplot(data=df, x=df.index, y='PIB_Indice', ax=axes[2], label='PIB (Índice de Volume)', color='#ff7f0e', linewidth=2)
    
    axes[2].set_title('Evolução do PIB (1999=100)', fontsize=16, fontweight='bold')
    axes[2].set_ylabel('Índice (1999=100)', fontsize=12)
    axes[2].set_xlabel('Ano', fontsize=10)
    axes[2].grid(True, alpha=0.3)
    
    # Configuração do Eixo X (Anos de 2 em 2)
    axes[2].xaxis.set_major_locator(mdates.YearLocator(2))
    axes[2].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.xticks(rotation=45)

    # Adicionar sombreados para os períodos de choque
    periodos_choque = [
        ('2002-06-01', '2003-01-01', 'Crise de Confiança\n(jun/2002-jan/2003)'),
        ('2008-09-01', '2009-03-01', 'Crise Financeira Global\n(set/2008-mar/2009)'),
        ('2015-01-01', '2016-12-01', 'Recessão Doméstica\n(jan/2015-dez/2016)'),
        ('2020-03-01', '2020-12-01', 'Pandemia Covid-19\n(mar/2020-dez/2020)')
    ]

    # Adicionar sombreado e atualizar legendas
    patch_crise = mpatches.Patch(color='gray', alpha=0.2, label='Choques Econômicos')

    for i, ax in enumerate(axes):
        # Adicionar sombreado
        for inicio, fim, label in periodos_choque:
            start = pd.to_datetime(inicio)
            end = pd.to_datetime(fim)
            ax.axvspan(start, end, color='gray', alpha=0.2)
            
            # Adicionar texto apenas no primeiro gráfico (axes[0])
            if i == 0:
                mid_point = start + (end - start) / 2
                # Posicionar texto no topo da área do gráfico
                ax.text(mid_point, 0.68, label, transform=ax.get_xaxis_transform(),
                        ha='center', va='top', fontsize=12, color='black',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', pad=2))
        
        # Atualizar legenda
        handles, labels = ax.get_legend_handles_labels()
        
        if i == 3:
            loc = 'upper right'
        elif i == 1:
            loc = 'upper right'
        elif i == 2:
            loc = 'lower right'
        else:
            loc = 'upper right'
            
        ax.legend(handles=handles + [patch_crise], loc=loc, frameon=True)

    plt.tight_layout()
    return fig

fig = plotar_dados(df_economico)
plt.show()

In [ ]:
# Exportar o gráfico para paper/graficos/series_macroeconomicas.png
# fig.savefig('/home/fernando/Documentos/dev/brazil-taylor-determinacy/paper/graficos/series_macroeconomicas.png', dpi=300)

In [ ]:
# df_economico.describe().to_latex('/home/fernando/Documentos/dev/brazil-taylor-determinacy/paper/tabelas/estatisticas_descritivas.tex', float_format="%.2f")

In [ ]:
# Exportar dados para CSV
df_economico.to_csv('/home/fernando/Documentos/dev/brazil-taylor-determinacy/dados/fernando/dados_economicos_consolidados.csv')